In [1]:
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import os
import csv

In [3]:
#activity labels as defined in activity_labels.txt
activity_labels = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING', 'STAND_TO_SIT', 'SIT_TO_STAND', 'SIT_TO_LIE', 'LIE_TO_SIT', 'STAND_TO_LIE', 'LIE_TO_STAND']
activity_labels = {k:v for k,v in enumerate(activity_labels, start=1)}

In [4]:
#load data
def load_data(path):
    data = pd.read_csv(path, header=None, delim_whitespace=True)
    return data.values

def load_set(path, x, y):
    data = load_data(path+x)
    labels = load_data(path+y)
    return data, labels

train_data, train_labels = load_set('HAPT Data Set/Train/', 'X_train.txt', 'y_train.txt')
test_data, test_labels = load_set('HAPT Data Set/Test/', 'X_test.txt', 'y_test.txt')

print(f'test_data before mod {test_data.shape}')
print(f'train_data before mod {train_data.shape}')

#reshape the data to add a features dimension (features = 1)
#https://stackoverflow.com/questions/43396572/dimension-of-shape-in-conv1d
train_data = np.expand_dims(train_data, axis=2)
test_data = np.expand_dims(test_data, axis=2)

print(f'test_data after mod {test_data.shape}')
print(f'train_data after mod {train_data.shape}')

test_data before mod (3162, 561)
train_data before mod (7767, 561)
test_data after mod (3162, 561, 1)
train_data after mod (7767, 561, 1)


In [5]:
#input shape
timesteps = train_data.shape[1] #561 timesteps
features = train_data.shape[2] #1 feature
print(train_labels.shape)

(7767, 1)


In [6]:
model = models.Sequential()
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu', input_shape=(timesteps,features)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Dropout(0.4))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256))
model.add(layers.Dense(128))
model.add(layers.Dense(64))
model.add(layers.Dense(32))
model.add(layers.Dense(13)) #not sure why the dense layer has to be 13 instead of the 12 activities


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 559, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 279, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 277, 64)           6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 138, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 136, 128)          24704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 68, 128)           0         
_________________________________________________________________
dropout (Dropout)            (None, 68, 128)           0

In [7]:
# Compiling the model
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [8]:
test_loss,test_acc = model.evaluate(test_data, test_labels, verbose=2)

3162/3162 - 2s - loss: 2.5455 - accuracy: 0.2198


In [15]:
pred_outs = model.predict_classes(test_data)

#test if the label matches the prediction
false_pred = 0
true_pred = 0
for i in range(len(pred_outs)):
    #print(f'test_label: {activity_labels[test_labels[i][0]]}\npredicted_label:{activity_labels[pred_outs[i]]}')
    if not (1 < pred_outs[i] or pred_outs[i] < 13):
        print('prediction out of bounds')
    if pred_outs[i]==test_labels[i][0]:
        #print('true')
        true_pred += 1
    else:
        #print('false')
        false_pred += 1
print(f'false predictions:{false_pred}')
print(f'true predictions:{true_pred}')
print(f'prediction accuracy: {true_pred/len(pred_outs)}')

false predictions:215
true predictions:2947
prediction accuracy: 0.9320050600885516


In [12]:
model.fit(train_data, 
          train_labels, 
          epochs=10, 
          validation_data=(test_data, test_labels))

test_loss,test_acc = model.evaluate( test_data, test_labels, verbose=2)

Train on 7767 samples, validate on 3162 samples
Epoch 1/10
7767/7767 [==============================] - 4s 577us/sample - loss: 0.6208 - accuracy: 0.7570 - val_loss: 0.2981 - val_accuracy: 0.8782
Epoch 2/10
7767/7767 [==============================] - 4s 459us/sample - loss: 0.1861 - accuracy: 0.9261 - val_loss: 0.3525 - val_accuracy: 0.8605
Epoch 3/10
7767/7767 [==============================] - 4s 457us/sample - loss: 0.1346 - accuracy: 0.9484 - val_loss: 0.3656 - val_accuracy: 0.8719
Epoch 4/10
7767/7767 [==============================] - 4s 467us/sample - loss: 0.1220 - accuracy: 0.9526 - val_loss: 0.2278 - val_accuracy: 0.9216
Epoch 5/10
7767/7767 [==============================] - 4s 455us/sample - loss: 0.0967 - accuracy: 0.9618 - val_loss: 0.1981 - val_accuracy: 0.9292
Epoch 6/10
7767/7767 [==============================] - 4s 474us/sample - loss: 0.0814 - accuracy: 0.9694 - val_loss: 0.4132 - val_accuracy: 0.8865
Epoch 7/10
7767/7767 [==============================] - 4s 472us